In [13]:
# Import the needed packages in the Python 3 environment
import pandas as pd
import numpy as np  # Importing the NumPy library for numerical operations and array manipulations
from IPython.display import display  # Import display for better output formatting

# Replace the URLs below with the raw URLs of your CSV files from GitHub
Athletes_file_url = 'https://raw.githubusercontent.com/cateallen/Marathon-Dynamics/refs/heads/main/data/Athletes.csv'
BQStandards_url = 'https://raw.githubusercontent.com/cateallen/Marathon-Dynamics/refs/heads/main/data/BQStandards.csv'
Weather_url = 'https://raw.githubusercontent.com/cateallen/Marathon-Dynamics/refs/heads/main/data/Weather.csv'

# Read the CSV files into DataFrames
data1 = pd.read_csv(Athletes_file_url)
data2 = pd.read_csv(BQStandards_url)
data3 = pd.read_csv(Weather_url)

# Display the first few rows of each DataFrame
print("Data from file 1:")
print(data1.head())

print("\nData from file 2:")
print(data2.head())

print("\nData from file 3:")
print(data3.head())

# Get average weather values for min, max, and mean temps for each zip code
weatherAgg = data3.groupby('Zip').agg(
    Min=('Min Temp', 'mean'),
    Max=('Max Temp', 'mean'),
    Mean=('Mean Temp', 'mean') 
).reset_index()

# Assign bin values based on average min temp
bins = [-20, 30, 40, 50, 60, np.inf]
values = ['< 30F', '30-39F', '40-49F', '50-59F', '>= 60F']
weatherAgg['Min Bins'] = pd.cut(weatherAgg['Min'], bins, labels=values)

# Load the Boston Qualifying (BQ) standards to see if the athlete was on pace to qualify at the halfway point during the race
athletes = pd.merge(left=data1, right=data2, left_on=['Gender', 'Age Group'], right_on=['Gender', 'Age Bracket'])
athletes['BQ Pace'] = athletes.apply(lambda x: True if x['First Half'] < x['Standard'] / 2 else False, axis=1)
athletes.drop('Standard', axis=1, inplace=True)

# Combine the dataframes into one results set that includes weather data and splits
results = pd.merge(left=athletes, right=weatherAgg, how='left')

# Data clean-up step, remove athletes that did not match with weather data
results = results.dropna()

# Make things pretty
results['Zip'] = results['Zip'].astype(str).str.pad(5, 'left', '0')
results['Min'] = results['Min'].round(2)
results['Max'] = results['Max'].round(2)
results['Mean'] = results['Mean'].round(2)
results['Percent Change'] = results['Percent Change'].round(4)

# Change the sort order so that it goes youngest category to older
results['Age Group'] = pd.Categorical(results['Age Group'], ["Under 35", "35-39", "40-44", "45-49", "50-54", "55-59", "60-64", "65-69", "70-74", "75-79", "80 and Over"])

# Display the results
display(results.info())
display(results.head())

Data from file 1:
   Bib    Zip  Age Age Group Gender  First Half  Second Half  Finish  \
0   25  93730   30  Under 35      M        3832         3961    7793   
1    9  97124   32  Under 35      M        3845         4059    7904   
2   20  80922   40     40-44      M        3795         4157    7952   
3   46   2136   38     35-39      M        3997         4065    8062   
4   51   6119   26  Under 35      M        3979         4198    8177   

   Positive Split  Percent Change  
0             129        0.033664  
1             214        0.055657  
2             362        0.095389  
3              68        0.017013  
4             219        0.055039  

Data from file 2:
  Gender Age Bracket  Standard
0      M    Under 35     10800
1      M       35-39     11100
2      M       40-44     11400
3      M       45-49     12000
4      M       50-54     12300

Data from file 3:
     Date  Max Temp  Min Temp  Mean Temp    Zip
0  4/1/24   66.2945   43.0745  53.263256  93730
1  4/2/24   7

None

,Bib,Zip,Age,Age Group,Gender,First Half,Second Half,Finish,Positive Split,Percent Change,Age Bracket,BQ Pace,Min,Max,Mean,Min Bins
0,25,93730,30,Under 35,M,3832,3961,7793,129,0.0337,Under 35,True,43.45,69.37,54.39,40-49F
1,9,97124,32,Under 35,M,3845,4059,7904,214,0.0557,Under 35,True,40.12,60.74,48.59,40-49F
2,51,06119,26,Under 35,M,3979,4198,8177,219,0.0550,Under 35,True,39.71,57.64,47.63,30-39F
3,36,90291,23,Under 35,M,3973,4215,8188,242,0.0609,Under 35,True,48.16,65.29,56.18,40-49F
4,43,72712,27,Under 35,M,3979,4249,8228,270,0.0679,Under 35,True,50.36,70.01,58.46,50-59F


In [51]:
display(pd.crosstab(results['Age Group'], results['Gender']))
display(pd.crosstab(results['Age Group'], [results['Gender'], results['BQ Pace']]))

Gender,F,M
Age Group,,
Under 35,2854,2433
35-39,1035,1223
40-44,1169,1162
45-49,1128,1155
50-54,764,975
55-59,572,836
60-64,389,707
65-69,154,409
70-74,41,138


Gender          F           M      
BQ Pace     False True  False True 
Age Group                          
Under 35     1333  1521   945  1488
35-39         412   623   494   729
40-44         462   707   413   749
45-49         378   750   420   735
50-54         304   460   362   613
55-59         206   366   309   527
60-64         126   263   240   467
65-69          51   103   126   283
70-74           9    32    57    81
75-79           4     2    13    20
80 and Over     1     0     2     5

In [17]:
# Create a histogram of the number of runners by minimum temp
subset = results[['Bib', 'Min']].copy()  # Create a copy to avoid SettingWithCopyWarning
subset['Min'] = subset['Min'].round(0).astype(int)  # Round and convert to int

# Group by 'Min' and count the number of 'Bib' entries
histo = subset.groupby('Min')['Bib'].count().reset_index(name='Finishers')

# Display the histogram DataFrame
display(histo)

/tmp/ipykernel_127/190469660.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Min'] = subset['Min'].round(0).astype(int)


,Min,Finishers
0,4,1
1,5,1
2,7,5
3,9,3
4,12,1
...,...,...
59,67,46
60,68,60
61,69,13
62,70,13


In [19]:
# Is there a correlation between time to complete the first half and the percent change in pace?
display(results[['First Half', 'Percent Change']].corr().iloc[0::2,-1])

First Half    0.114274
Name: Percent Change, dtype: float64

In [23]:
import pandas as pd
from tabulate import tabulate  # Ensure you have this import

# Disaggregate the correlations by gender and age group
corrs = results.groupby(['Gender', 'Age Group'], observed=False)[['First Half', 'Percent Change']].corr().reset_index()

# Filter for 'First Half' correlations
corrs = corrs.loc[corrs['level_2'] == 'First Half']

# Exclude certain age groups
corrs = corrs.loc[~corrs['Age Group'].isin(['70-74', '75-79', '80 and Over'])]

# Round the 'Percent Change' values
corrs['Percent Change'] = corrs['Percent Change'].round(3)

# Replace gender codes with full names
corrs['Gender'] = corrs['Gender'].replace({'M': 'Men', 'F': 'Women'})

# Pivot the DataFrame
corrs_pivot = corrs.pivot(index='Age Group', columns='Gender', values='Percent Change')

# Print the pivoted DataFrame using tabulate
print(tabulate(corrs_pivot, headers='keys', tablefmt='psql'))

+-------------+-------+---------+
| Age Group   |   Men |   Women |
|-------------+-------+---------|
| Under 35    | 0.148 |   0.097 |
| 35-39       | 0.2   |   0.16  |
| 40-44       | 0.217 |   0.122 |
| 45-49       | 0.205 |   0.185 |
| 50-54       | 0.19  |   0.196 |
| 55-59       | 0.168 |   0.199 |
| 60-64       | 0.066 |   0.113 |
| 65-69       | 0.176 |   0.069 |
+-------------+-------+---------+


/tmp/ipykernel_127/9868386.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  corrs = results.groupby(['Gender', 'Age Group'])[['First Half', 'Percent Change']].corr().reset_index()


In [25]:
# Is there a correlation between the min temp and the change in pace?
# subset = results.loc[(results['First Half'] < 5400)]
corrs = results.groupby(['Gender', 'Age Group'])[['Min', 'Percent Change']].corr().reset_index()
corrs = corrs.loc[corrs['level_2'] == 'Min']
corrs = corrs.loc[~corrs['Age Group'].isin(['70-74', '75-79', '80 and Over'])]
corrs['Percent Change'] = corrs['Percent Change'].round(3)
corrs['Gender'] = corrs['Gender'].replace({'M' : 'Men', 'F' : 'Women'})
corrs = pd.pivot(data=corrs, index='Age Group', columns='Gender', values='Percent Change')

print(tabulate(corrs, headers='keys', tablefmt='psql'))

+-------------+--------+---------+
| Age Group   |    Men |   Women |
|-------------+--------+---------|
| Under 35    | -0.052 |  -0.071 |
| 35-39       | -0.001 |   0.024 |
| 40-44       | -0.043 |   0.022 |
| 45-49       |  0.02  |  -0.013 |
| 50-54       |  0.008 |  -0.084 |
| 55-59       |  0.003 |   0.024 |
| 60-64       |  0.055 |   0.013 |
| 65-69       |  0.023 |   0.131 |
+-------------+--------+---------+


/tmp/ipykernel_127/3745968853.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  corrs = results.groupby(['Gender', 'Age Group'])[['Min', 'Percent Change']].corr().reset_index()


In [27]:
# Is there a correlation between the max temp and the change in pace?
corrs = results.groupby(['Gender', 'Age Group'], observed=False)[['Max', 'Percent Change']].corr().reset_index()

# Filter for 'Max' correlations
corrs = corrs.loc[corrs['level_2'] == 'Max']

# Exclude certain age groups
corrs = corrs.loc[~corrs['Age Group'].isin(['70-74', '75-79', '80 and Over'])]

# Round the 'Percent Change' values
corrs['Percent Change'] = corrs['Percent Change'].round(3)

# Replace gender codes with full names
corrs['Gender'] = corrs['Gender'].replace({'M': 'Men', 'F': 'Women'})

# Pivot the DataFrame
corrs = pd.pivot(data=corrs, index='Age Group', columns='Gender', values='Percent Change')

# Print the pivoted DataFrame using tabulate
print(tabulate(corrs, headers='keys', tablefmt='psql'))

+-------------+--------+---------+
| Age Group   |    Men |   Women |
|-------------+--------+---------|
| Under 35    | -0.086 |  -0.083 |
| 35-39       | -0.023 |   0.005 |
| 40-44       | -0.059 |   0.005 |
| 45-49       |  0.003 |  -0.045 |
| 50-54       | -0.006 |  -0.073 |
| 55-59       | -0.019 |   0.019 |
| 60-64       |  0.061 |  -0.008 |
| 65-69       |  0.056 |   0.148 |
+-------------+--------+---------+


/tmp/ipykernel_127/1511378325.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  corrs = results.groupby(['Gender', 'Age Group'])[['Max', 'Percent Change']].corr().reset_index()


In [29]:
# Is there a correlation between mean temperature and the change in pace?
corrs = results.groupby(['Gender', 'Age Group'], observed=False)[['Mean', 'Percent Change']].corr().reset_index()

# Filter for 'Mean' correlations
corrs = corrs.loc[corrs['level_2'] == 'Mean']

# Exclude certain age groups
corrs = corrs.loc[~corrs['Age Group'].isin(['70-74', '75-79', '80 and Over'])]

# Pivot the DataFrame
corrs = pd.pivot(data=corrs, index='Age Group', columns='Gender', values='Percent Change')

# Print the pivoted DataFrame using tabulate
print(tabulate(corrs, headers='keys', tablefmt='psql'))

+-------------+------------+-------------+
| Age Group   |          F |           M |
|-------------+------------+-------------|
| Under 35    | -0.0795701 | -0.0710745  |
| 35-39       |  0.015774  | -0.0109444  |
| 40-44       |  0.0165575 | -0.0495545  |
| 45-49       | -0.0301416 |  0.0143096  |
| 50-54       | -0.0826197 |  0.0057196  |
| 55-59       |  0.0218601 | -0.00768541 |
| 60-64       |  0.0027626 |  0.0599705  |
| 65-69       |  0.14295   |  0.0410385  |
+-------------+------------+-------------+


/tmp/ipykernel_127/2736796703.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  corrs = results.groupby(['Gender', 'Age Group'])[['Mean', 'Percent Change']].corr().reset_index()


In [49]:
# How does the median change vary with gender, age group, and min temp?
median = results.groupby(['Gender', 'Age Group', 'Min Bins'], observed=False)['Percent Change'].median().reset_index()

# Exclude certain age groups
median = median.loc[~median['Age Group'].isin(['70-74', '75-79', '80 and Over'])]

# Round the 'Percent Change' values
median['Percent Change'] = median['Percent Change'].round(3)

# Print the output for men
men = pd.pivot(median.loc[median['Gender'] == 'M'], index='Age Group', columns='Min Bins', values='Percent Change')
print(tabulate(men, headers='keys', tablefmt='psql'))
print()

# Print the output for women
women = pd.pivot(median.loc[median['Gender'] == 'F'], index='Age Group', columns='Min Bins', values='Percent Change')
print(tabulate(women, headers='keys', tablefmt='psql'))
print()

# Prepare the data for an output file to create a visualization
output = pd.pivot(median, index=['Gender', 'Age Group'], columns='Min Bins', values='Percent Change').reset_index()
output['Gender'] = output['Gender'].replace({'M': 'Men', 'F': 'Women'})

print(tabulate(output, headers='keys', tablefmt='psql'))

+-------------+---------+----------+----------+----------+----------+
| Age Group   |   < 30F |   30-39F |   40-49F |   50-59F |   >= 60F |
|-------------+---------+----------+----------+----------+----------|
| Under 35    |   0.133 |    0.167 |    0.133 |    0.139 |    0.141 |
| 35-39       |   0.166 |    0.146 |    0.12  |    0.152 |    0.121 |
| 40-44       |   0.095 |    0.149 |    0.127 |    0.134 |    0.115 |
| 45-49       |   0.136 |    0.167 |    0.145 |    0.158 |    0.148 |
| 50-54       |   0.087 |    0.153 |    0.142 |    0.139 |    0.165 |
| 55-59       |   0.141 |    0.18  |    0.146 |    0.143 |    0.143 |
| 60-64       |   0.174 |    0.156 |    0.167 |    0.183 |    0.219 |
| 65-69       |   0.181 |    0.19  |    0.174 |    0.183 |    0.129 |
+-------------+---------+----------+----------+----------+----------+

+-------------+---------+----------+----------+----------+----------+
| Age Group   |   < 30F |   30-39F |   40-49F |   50-59F |   >= 60F |
|-------------+----

In [33]:
# Filter to only show men under 35 and export data for scatter plot
subset = results.loc[(results['Gender'] == 'M') & (results['Age Group'] == 'Under 35')]
subset = subset[['First Half', 'Percent Change']]
subset['First Half'] = pd.to_datetime(subset['First Half'], unit='s').dt.strftime('%H:%M:%S')

In [47]:
# Create histogram of runners by percent change in pace
subset = results[['Bib', 'Percent Change']]

bins = [-np.inf, 0, 0.05, 0.10, 0.15, 0.20, 0.25, np.inf]
values = ['Negative', '0 to 5%', '5% to 10%', '10% to 15%', '15% to 20%', '20% to 25%', 'Over 25%']

# Subset['Percent Bins'] = np.select(conditions, values)
subset['Percent Bins'] = pd.cut(subset['Percent Change'], bins, labels=values, include_lowest=True)

# Apply observed=False to groupby
output = subset.groupby('Percent Bins', observed=False)['Bib'].count().reset_index(name='Finishers')

display(output)

/tmp/ipykernel_127/1087328543.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Percent Bins'] = pd.cut(subset['Percent Change'], bins, labels=values, include_lowest=True)


,Percent Bins,Finishers
0,Negative,504
1,0 to 5%,2325
2,5% to 10%,3456
3,10% to 15%,3307
4,15% to 20%,2660
5,20% to 25%,1809
6,Over 25%,3130


In [45]:
subset = results.groupby(['BQ Pace', 'Gender', 'Min Bins'], observed=False)['Percent Change'].median().reset_index(name='Change')
display(pd.pivot(subset, index='Min Bins', columns=['Gender', 'BQ Pace'], values='Change'))

Gender,F,M,F,M
BQ Pace,False,False,True,True
Min Bins,,,,
< 30F,0.1442,0.17190,0.09770,0.12005
30-39F,0.1446,0.19610,0.10400,0.12430
40-49F,0.1426,0.18670,0.10310,0.12435
50-59F,0.1620,0.21040,0.10715,0.12345
>= 60F,0.1475,0.19505,0.10875,0.12230


In [39]:
corrs = results.groupby(['Gender', 'BQ Pace'])[['First Half', 'Percent Change']].corr().reset_index()
corrs = corrs.loc[corrs['level_2'] == 'First Half']
corrs['Percent Change'] = corrs['Percent Change'].round(3)
corrs['Gender'] = corrs['Gender'].replace({'M' : 'Men', 'F' : 'Women'})
corrs = pd.pivot(data=corrs, index='BQ Pace', columns='Gender', values='Percent Change')

print(tabulate(corrs, headers='keys', tablefmt='psql'))

+-----------+--------+---------+
| BQ Pace   |    Men |   Women |
|-----------+--------+---------|
| False     | -0.014 |  -0.034 |
| True      |  0.138 |   0.149 |
+-----------+--------+---------+


In [41]:
corrs = results.groupby(['Gender', 'BQ Pace'])[['Min', 'Percent Change']].corr().reset_index()
corrs = corrs.loc[corrs['level_2'] == 'Min']
corrs['Percent Change'] = corrs['Percent Change'].round(3)
corrs['Gender'] = corrs['Gender'].replace({'M' : 'Men', 'F' : 'Women'})
corrs = pd.pivot(data=corrs, index='BQ Pace', columns='Gender', values='Percent Change')

print(tabulate(corrs, headers='keys', tablefmt='psql'))

+-----------+-------+---------+
| BQ Pace   |   Men |   Women |
|-----------+-------+---------|
| False     | 0.02  |   0.054 |
| True      | 0.011 |  -0.011 |
+-----------+-------+---------+
